# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#create import file path
inputFile = "../WeatherPy/Output/cities.csv"

#load dataframe
weatherCitiesDF = pd.read_csv(inputFile,keep_default_na=False)

#create output file

outputPath="../Output/"
# outputCsv= outputPath+"cities.csv"

weatherCitiesDF.loc[(weatherCitiesDF['City'] == 'Gobabis')].head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
182,Gobabis,-22.45,18.9667,76.33,14,0,4.41,NA,1619269721


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# List for holding cities
gmaps.configure(api_key=g_key)

#initiate the figure
fig = gmaps.figure()

#get data from dataframe
locations = weatherCitiesDF[["Lat", "Lng"]]
heat = weatherCitiesDF["Humidity"].astype(int)

#create heat layer
heat_layer = gmaps.heatmap_layer(locations
                                 ,weights=heat
                                 ,dissipating=False
                                 ,max_intensity=15
                                 ,point_radius=2)

#add layer
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#create new dataframe based on the conditions in instructions
niceWeatherDF = weatherCitiesDF.loc[(weatherCitiesDF["Max Temp"].between(70,80))
                                    & (weatherCitiesDF["Wind Speed"] < 10.00)
                                    & (weatherCitiesDF["Cloudiness"] == 0) 
                                   ]
#remove NAs
niceWeatherDF.dropna(axis=0)
niceWeatherDF.describe() #sanity check

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,6.000000,6.000000,6.000000,6.000000,6.0,6.000000,6.000000e+00
mean,-15.506450,55.932083,75.361667,48.666667,0.0,4.710000,1.619270e+09
std,23.484573,46.948071,3.044637,28.619341,0.0,3.025961,2.507410e+02
min,-33.015300,18.966700,71.600000,14.000000,0.0,1.010000,1.619269e+09
25%,-24.650025,23.290400,73.400000,24.500000,0.0,2.677500,1.619270e+09
50%,-23.225000,31.337900,74.865000,53.000000,0.0,4.510000,1.619270e+09
75%,-20.850025,93.916025,77.590000,68.750000,0.0,6.335000,1.619270e+09
max,31.610000,118.566700,79.430000,83.000000,0.0,9.220000,1.619270e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
nameL = []
latL = []
lngL = []
addyL = []
cityL = []
ctryL = []

for index, row in niceWeatherDF.iterrows():
    try:
        #cap lat and lng for city
        lat = row['Lat']
        lng = row['Lng']
    #     print(city) #sanity check
    #     print(f"Lat: {lat}") #sanity check
    #     print(f"Lng: {lng}") #sanity check

        #geocoordinates
        targetCoordinates = f"{lat} , {lng}"
        targetSearch = "hotel"
        targetRadius = 5000
        #targetType = "hotel"
        #print(targetCoordinates) #sanity check

        # set up a parameters dictionary
        params = {
            "location": targetCoordinates,
            "keyword": targetSearch,
            "radius": targetRadius,
            #"type": targetType,
            "key": g_key
            }

        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        #get results
        response = requests.get(base_url, params=params).json()
        
        nameL.append(response['results'][0]['name'])
        latL.append(response['results'][0]['geometry']['location']['lat'])
        lngL.append(response['results'][0]['geometry']['location']['lng'])
        addyL.append(response['results'][0]['vicinity'])
        cityL.append(row['City'])
        ctryL.append(row['Country'])
        
    except KeyError:
        print(f"City or Hotel not found")
      

In [6]:
hotelDict = {
    "Hotel Name" : nameL,
    "City" : cityL,
    "Hotel Lat" : latL,
    "Hotel Lng" : lngL,
    "Country" : ctryL,
    "Address" : addyL
} 

#convert dictionary to df
hotelDF = pd.DataFrame(hotelDict)

hotelDF

,Hotel Name,City,Hotel Lat,Hotel Lng,Country,Address
0,Hospitality Carnarvon,Carnarvon,-24.892341,113.655034,AU,"6 West St, South Carnarvon"
1,Blue Lagoon Hotel,East London,-32.983472,27.951098,ZA,"21 Blue Bend Pl, Blue Bend, East London"
2,Onze Rust Guest House,Gobabis,-22.454125,18.977207,NA,"Rugby, Gobabis"
3,The Esplanade Hotel,Port Hedland,-20.313925,118.576081,AU,"2 Anderson St, Port Hedland"
4,Entabeni Hotel,Hukuntsi,-23.999329,21.791398,BW,"Unnamed Road, Hukuntsi"
5,Desert Gat,Kiryat Gat,31.602743,34.764339,IL,"Sderot HaAtsma'ut 134, Kiryat Gat"


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
infoBoxTemplate = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Address</dt><dd>{Address}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotelInfo = [infoBoxTemplate.format(**row) for index, row in hotelDF.iterrows()]
locations = hotelDF[["Hotel Lat", "Hotel Lng"]]

In [9]:
# Add marker layer ontop of heat map
fig 

markers = gmaps.marker_layer(locations, 
            info_box_content=hotelInfo)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))